# notebooks/pulse\_score.ipynb

**Overview**
This notebook ingests Buffalo raw data from Athena, spatially joins to 2020 Census tracts, computes tract-level metrics (crime, vacancy, permits, licences, 311), derives a composite score, and stores results for visualization and LLM narration.

---

## 1. Setup & Imports


In [ ]:
import json
import os
import awswrangler as wr
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import shap

/Users/colinspear/dev/civic-pulse-rochester/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



Configure AWS:


In [2]:
os.environ['AWS_REGION'] = 'us-east-1'
wr.config.athena_workgroup = 'primary'
DATABASE = 'civic_pulse'


---

## 2. Load Raw Tables from Athena


In [13]:
# helper: read partition for a specific date
def read_last_n_days(table, n=7):
    query = f"""
      SELECT *
      FROM {DATABASE}.{table}
      WHERE pulled_utc >= date_add('day', -{n}, current_timestamp)
    """
    return wr.athena.read_sql_query(query, database=DATABASE)

# example: 7-day window ending today
yesterday = pd.Timestamp.utcnow().normalize() - pd.Timedelta(days=1)
y, m, d = yesterday.year, f"{yesterday.month:02}", f"{yesterday.day:02}"

crime_df = read_last_n_days('raw_buf_crime')
viol_df  = read_last_n_days('raw_buf_viol')
perm_df  = read_last_n_days('raw_buf_permits')
biz_df   = read_last_n_days('raw_buf_biz')
calls_df = read_last_n_days('raw_buf_311')


---

## 3. Load 2020 Tract Shapefile


In [51]:
# NY State tracts (state FIPS 36) from TIGER 2024
url = ("https://www2.census.gov/geo/tiger/TIGER2024/TRACT/"
       "tl_2024_36_tract.zip")

tracts = gpd.read_file(url)[["GEOID", "geometry"]].to_crs(epsg=4326)
tracts = tracts[tracts["GEOID"].str.startswith("36029")].copy()

In [ ]:
path = "../data/acs2023_erie_tract_pop.json"   # adjust as needed

with open(path) as f:
    rows = json.load(f)        # rows[0] = header list, rows[1:] = data

header, data = rows[0], rows[1:]
acs = pd.DataFrame(data, columns=header)

# numeric casts
acs["population"]     = pd.to_numeric(acs["B02001_001E"], errors="coerce")
acs["housing_units"]  = pd.to_numeric(acs["B25001_001E"], errors="coerce")

# 11-digit GEOID
acs["GEOID"] = (
    acs["state"].str.zfill(2) +
    acs["county"].str.zfill(3) +
    acs["tract"].str.zfill(6)
)

acs = acs[["GEOID", "population", "housing_units"]]

tracts = tracts.merge(acs, on="GEOID", how="left")

---

## 4. Spatial Join Points → Tracts


In [ ]:
# TODO: Add geocoder to extract lat/lon from addresses in permitting data
# import requests, pandas as pd, time, json

# def geocode_census(addr_series, batch=100):
#     out = {}
#     for chunk in addr_series.groupby(addr_series.index // batch):
#         payload = "\n".join(chunk[1].tolist())
#         resp = requests.post(
#             "https://geocoding.geo.census.gov/geocoder/geographies/addressbatch",
#             files={"addressFile": ("addrs.txt", payload),
#                    "benchmark": (None, "Public_AR_Current"),
#                    "vintage": (None, "Current_Current")}
#         )
#         for line in resp.text.strip().split("\n"):
#             parts = line.split(",")
#             if len(parts) >= 9 and parts[8]:
#                 out[parts[0]] = parts[8].zfill(11)   # tract GEOID
#         time.sleep(0.2)   # stay polite
#     return pd.Series(out, name="tract")

# perm_df["uid"] = perm_df.index.astype(str)  # unique row key for batch file
# perm_df["tract"] = geocode_census(perm_df["address"])

def join_points(df, lon='longitude', lat='latitude'):
    """
    If lon/lat columns are present, spatially join to tracts.
    Otherwise just add a null 'tract' column and return the original df.
    """
    if lon in df.columns and lat in df.columns:
        gdf = gpd.GeoDataFrame(
            df,
            geometry=gpd.points_from_xy(df[lon], df[lat]),
            crs='EPSG:4326'
        )
        out = gpd.sjoin(gdf, tracts[['GEOID', 'geometry']],
                        how='left', predicate='within')
        return out.drop(columns='geometry').rename(columns={'GEOID': 'tract'})
    else:
        df = df.copy()
        df['tract'] = pd.NA
        return df

crime_gdf = join_points(crime_df)
viol_gdf  = join_points(viol_df)
perm_gdf  = join_points(perm_df, lon=None, lat=None)  # if no coords
biz_gdf   = join_points(biz_df)
calls_gdf = join_points(calls_df)


---

## 5. Compute Tract-Level Metrics


In [57]:
# initialise metrics with all tracts
metrics = pd.DataFrame({
    "tract": tracts["GEOID"],
    "population": tracts["population"],
    "housing_units": tracts["housing_units"]
})


In [58]:

# 5.1 crime per 1 000 residents
crime_counts = crime_gdf.groupby("tract").size().rename("crime_count")
metrics = metrics.merge(crime_counts, left_on="tract", right_index=True, how="left").fillna(0)

metrics["crime_per_1k"] = metrics["crime_count"] / (metrics["population"] / 1000).replace({0: pd.NA})

# 5.2 vacancy violations
viol_counts = viol_gdf.groupby("tract").size().rename("vacant_code_count")
metrics = metrics.merge(viol_counts, left_on="tract", right_index=True, how="left").fillna(0)

# 5.3 permits (until geocoded, these stay NaN -> will be ignored in z-scores)
perm_counts = perm_gdf.groupby("tract").size().rename("permit_count")
metrics = metrics.merge(perm_counts, left_on="tract", right_index=True, how="left").fillna(0)

# 5.4 business licences
biz_counts = biz_gdf.groupby("tract").size().rename("licence_count")
metrics = metrics.merge(biz_counts, left_on="tract", right_index=True, how="left").fillna(0)

# 5.5 311 calls
call_counts = calls_gdf.groupby("tract").size().rename("calls_count")
metrics = metrics.merge(call_counts, left_on="tract", right_index=True, how="left").fillna(0)

metrics.head()

/var/folders/tj/2zm1pzk90t177_g7ndj99_y00000gn/T/ipykernel_6212/75387334.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  metrics = metrics.merge(viol_counts, left_on="tract", right_index=True, how="left").fillna(0)


,tract,population,housing_units,crime_count,crime_per_1k,vacant_code_count,permit_count,licence_count,calls_count
0,36029980500,737,15,44.0,59.701493,0.0,0.0,0.0,100.0
1,36029004601,3610,1563,102.0,28.254848,0.0,0.0,0.0,1639.0
2,36029010902,4205,2484,0.0,0.000000,0.0,0.0,0.0,0.0
3,36029009008,4753,2646,0.0,0.000000,0.0,0.0,0.0,0.0
4,36029009501,5107,2051,0.0,0.000000,0.0,0.0,0.0,0.0


In [66]:
metrics.head()

,tract,population,housing_units,crime_count,crime_per_1k,vacant_code_count,permit_count,licence_count,calls_count
0,36029980500,737,15,44.0,59.701493,0.0,0.0,0.0,100.0
1,36029004601,3610,1563,102.0,28.254848,0.0,0.0,0.0,1639.0
2,36029010902,4205,2484,0.0,0.000000,0.0,0.0,0.0,0.0
3,36029009008,4753,2646,0.0,0.000000,0.0,0.0,0.0,0.0
4,36029009501,5107,2051,0.0,0.000000,0.0,0.0,0.0,0.0


---

## 6. Clean and calculate z-scores

In [67]:
features = ['crime_per_1k','vacant_code_count','permit_count','licence_count','calls_count']

# 0. ensure numeric, replace NaN, drop zero-variance cols
metrics[features] = metrics[features].fillna(0)
features = [f for f in features if metrics[f].std() > 0]

# 1. z-score each feature then sum
scaler = StandardScaler()
metrics[features] = scaler.fit_transform(metrics[features])
metrics["score"]  = metrics[features].sum(axis=1)


---
## 7. Build weekly dataset

In [74]:
metrics.columns

Index(['tract', 'population', 'housing_units', 'crime_count', 'crime_per_1k',
       'vacant_code_count', 'permit_count', 'licence_count', 'calls_count',
       'score'],
      dtype='object')

In [79]:
calls_gdf["week"] = (
    calls_gdf["createddate"].dt.to_period("W-SUN").dt.start_time
)

# calls_next_week = size grouped by tract+week, shifted forward
weekly_y = (
    calls_gdf.groupby(["tract", "week"]).size()
             .rename("calls_next_week")
             .groupby(level=0).shift(-1)     # t+1
             .reset_index(drop=False)
)

# merge tract-level features with target
weekly_X = (
    metrics.merge(weekly_y, on="tract")
           .dropna(subset=["calls_next_week"])
)

# predictor list (no calls_count)
feature_cols = [
    "crime_per_1k",
    "vacant_code_count",
    "permit_count",
    "licence_count"
]

# fill any remaining NaN in predictors with 0
weekly_X[feature_cols] = weekly_X[feature_cols].fillna(0)

---

## 8. XGBoost & SHAP

In [85]:
test_weeks = 1
cutoff = weekly_X["week"].max() - pd.Timedelta(weeks=test_weeks)
train  = weekly_X[weekly_X.week <  cutoff]
test   = weekly_X[weekly_X.week >= cutoff]

X_train = train[feature_cols].to_numpy()
y_train = train["calls_next_week"]
X_test  = test[feature_cols].to_numpy()
y_test  = test["calls_next_week"]


In [87]:
train.shape

(178, 12)

In [88]:

model = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    objective="reg:squarederror",
)
model.fit(X_train, y_train)

explainer   = shap.Explainer(model, X_train, feature_names=feature_cols)
shap_values = explainer(X_test)          # additivity check now passes

In [89]:
shap_values

.values =
array([[-6.14082149e+01, -4.66870159e+00,  0.00000000e+00,
         1.66395179e+00],
       [ 1.78570650e+01, -1.42286087e+01,  0.00000000e+00,
         1.85226023e+00],
       [ 1.78570650e+01, -1.42286087e+01,  0.00000000e+00,
         1.85226023e+00],
       [ 5.62970747e+01, -1.32824527e+01,  0.00000000e+00,
         1.87845156e+00],
       [ 5.62970747e+01, -1.32824527e+01,  0.00000000e+00,
         1.87845156e+00],
       [-5.98670249e+00, -1.55416875e+01,  0.00000000e+00,
         1.75366515e+00],
       [-5.98670249e+00, -1.55416875e+01,  0.00000000e+00,
         1.75366515e+00],
       [-3.16074677e+01, -4.63320420e+00,  0.00000000e+00,
         1.60875834e+00],
       [-3.16074677e+01, -4.63320420e+00,  0.00000000e+00,
         1.60875834e+00],
       [-8.10014060e+00, -1.42286087e+01,  0.00000000e+00,
         1.91873361e+00],
       [-8.10014060e+00, -1.42286087e+01,  0.00000000e+00,
         1.91873361e+00],
       [ 1.43653603e+02, -1.42286087e+01,  0.00000000e+

---

## 8. Save Results to S3

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import boto3

metrics['run_date'] = pd.Timestamp.utcnow()

# write Parquet locally or to S3
table = pa.Table.from_pandas(metrics)
buf = pa.BufferOutputStream()
pq.write_table(table, buf, compression='zstd')
key = f"analytics/buf_pulse_score/{y}/{m}/{d}/pulse_score.parquet"

boto3.client('s3').put_object(
    Bucket=os.getenv('BUCKET'),
    Key=key,
    Body=buf.getvalue().to_pybytes()
)
print("Wrote composite metrics → s3://{}/{}".format(os.getenv('BUCKET'),key))



---

*Next:* Fill in ACS population load, training data for SHAP, and refine model fitting.
